In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import joblib
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths
GEN_A2B_INPUT_PATH = r"C:\\Users\\cl502_11\\MG\\Feature Extraction\\Latents for 2nd Model\\INPUT\\mel_spectrograms\\MW.npy"
GEN_B2A_INPUT_PATH = r"C:\\Users\\cl502_11\\MG\\Feature Extraction\\GANs Inputs\\Generator\\MW_latent.npy"
DISC_REAL_A_DIR = r"C:\\Users\\cl502_11\\MG\\Feature Extraction\\DataChunk1 (29 Files)\\80_10_10\\mel_spectrograms\\train"
DISC_REAL_B_DIR = r"C:\\Users\\cl502_11\\MG\\Feature Extraction\\GANs Inputs\\Discriminator"
SAVE_DIR = r"C:\Users\cl502_11\MG\Models\CycleGANs"
os.makedirs(SAVE_DIR, exist_ok=True)

# Hyperparameters
latent_dim = 128
epochs = 30
batch_size = 8
learning_rate = 0.0001

# Dummy Generator and Discriminator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        return self.model(x)

# Dataset class
class NumpyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Load data
input_data_A2B = np.load(GEN_A2B_INPUT_PATH)
input_data_B2A = np.load(GEN_B2A_INPUT_PATH)

# Flatten if needed
input_data_A2B = input_data_A2B.reshape(input_data_A2B.shape[0], -1)
input_data_B2A = input_data_B2A.reshape(input_data_B2A.shape[0], -1)

input_dim = input_data_A2B.shape[1]

# Dataloaders
dataset_A2B = NumpyDataset(torch.tensor(input_data_A2B, dtype=torch.float32))
data_loader_A2B = DataLoader(dataset_A2B, batch_size=batch_size, shuffle=True)

# Models
gen = Generator(input_dim=input_dim, output_dim=input_dim).to(device)
disc = Discriminator(input_dim=input_dim).to(device)

# Loss and Optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(gen.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(disc.parameters(), lr=learning_rate)

# Trackers
g_losses = []
d_losses = []

# Training loop
for epoch in range(epochs):
    for real_data in tqdm(data_loader_A2B, desc=f"Epoch {epoch+1}/{epochs}"):
        real_data = real_data.to(device)

        batch_size_curr = real_data.size(0)
        real_label = torch.ones((batch_size_curr, 1), device=device)
        fake_label = torch.zeros((batch_size_curr, 1), device=device)

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Real data
        output_real = disc(real_data)
        loss_real = criterion(output_real, real_label)

        # Fake data
        noise = torch.randn((batch_size_curr, input_dim), device=device)
        fake_data = gen(noise)
        output_fake = disc(fake_data.detach())
        loss_fake = criterion(output_fake, fake_label)

        # Total discriminator loss
        d_loss = loss_real + loss_fake
        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()
        output_fake_for_g = disc(fake_data)
        g_loss = criterion(output_fake_for_g, real_label)
        g_loss.backward()
        optimizer_G.step()

        # Track losses
        d_losses.append(d_loss.item())
        g_losses.append(g_loss.item())

    print(f"Epoch [{epoch+1}/{epochs}] - D Loss: {d_loss.item():.4f} - G Loss: {g_loss.item():.4f}")

# Save models
torch.save(gen.state_dict(), os.path.join(SAVE_DIR, "generator.pth"))
torch.save(disc.state_dict(), os.path.join(SAVE_DIR, "discriminator.pth"))

# -----------------------
# Evaluation: Loss Graph
# -----------------------
plt.figure(figsize=(10, 5))
plt.plot(g_losses, label="Generator Loss", color='green')
plt.plot(d_losses, label="Discriminator Loss", color='red')
plt.title("Generator vs Discriminator Loss")
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "loss_curve.png"))
plt.show()



Using device: cpu


Epoch 1/30: 100%|██████████| 16/16 [00:00<00:00, 23.68it/s]


Epoch [1/30] - D Loss: 0.7942 - G Loss: 0.6229


Epoch 2/30: 100%|██████████| 16/16 [00:00<00:00, 26.58it/s]


Epoch [2/30] - D Loss: 2.1252 - G Loss: 0.9739


...

Epoch 29/30: 100%|██████████| 16/16 [00:00<00:00, 33.54it/s]


Epoch [29/30] - D Loss: 1.2929 - G Loss: 0.3113


Epoch 30/30: 100%|██████████| 16/16 [00:00<00:00, 31.99it/s]


Epoch [30/30] - D Loss: 3.9162 - G Loss: 0.0214


: 

In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Paths and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
SAVE_DIR = r"C:\Users\cl502_11\MG\Models\CycleGANs"
os.makedirs(SAVE_DIR, exist_ok=True)

# Hyperparameter search space
learning_rates = [0.0001, 0.0002, 0.0005, 0.001]
batch_sizes = [8, 16, 32, 64]
latent_dims = [64, 128, 256]
epochs_options = [30, 50, 70, 100 ]

# Dummy Generator and Discriminator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        return self.model(x)

# Dataset class
class NumpyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Load data
GEN_A2B_INPUT_PATH = r"C:\\Users\\cl502_11\\MG\\Feature Extraction\\Latents for 2nd Model\\INPUT\\mel_spectrograms\\MW.npy"
input_data_A2B = np.load(GEN_A2B_INPUT_PATH)
input_data_A2B = input_data_A2B.reshape(input_data_A2B.shape[0], -1)

# DataLoader
input_dim = input_data_A2B.shape[1]
dataset_A2B = NumpyDataset(torch.tensor(input_data_A2B, dtype=torch.float32))
data_loader_A2B = DataLoader(dataset_A2B, batch_size=16, shuffle=True)

# Random Search
best_g_loss = float('inf')
best_d_loss = float('inf')
best_params = None
best_model = None

# Search over hyperparameter space
for lr in learning_rates:
    for batch_size in batch_sizes:
        for latent_dim in latent_dims:
            for epochs in epochs_options:
                
                print(f"Running Random Search for: lr={lr}, batch_size={batch_size}, latent_dim={latent_dim}, epochs={epochs}")
                
                # Initialize models
                gen = Generator(input_dim=latent_dim, output_dim=input_dim).to(device)
                disc = Discriminator(input_dim=input_dim).to(device)
                
                # Loss and Optimizers
                criterion = nn.BCELoss()
                optimizer_G = optim.Adam(gen.parameters(), lr=lr)
                optimizer_D = optim.Adam(disc.parameters(), lr=lr)

                # Track losses
                g_losses = []
                d_losses = []

                # Training loop
                for epoch in range(epochs):
                    for real_data in tqdm(data_loader_A2B, desc=f"Epoch {epoch+1}/{epochs}"):
                        real_data = real_data.to(device)

                        batch_size_curr = real_data.size(0)
                        real_label = torch.ones((batch_size_curr, 1), device=device)
                        fake_label = torch.zeros((batch_size_curr, 1), device=device)

                        # --------------------- Train Discriminator ---------------------
                        optimizer_D.zero_grad()

                        # Real data
                        output_real = disc(real_data)
                        loss_real = criterion(output_real, real_label)

                        # Fake data
                        noise = torch.randn((batch_size_curr, latent_dim), device=device)
                        fake_data = gen(noise)
                        output_fake = disc(fake_data.detach())
                        loss_fake = criterion(output_fake, fake_label)

                        # Total discriminator loss
                        d_loss = loss_real + loss_fake
                        d_loss.backward()
                        optimizer_D.step()

                        # ----------------- Train Generator -----------------
                        optimizer_G.zero_grad()
                        output_fake_for_g = disc(fake_data)
                        g_loss = criterion(output_fake_for_g, real_label)
                        g_loss.backward()
                        optimizer_G.step()

                        # Track losses
                        d_losses.append(d_loss.item())
                        g_losses.append(g_loss.item())

                    print(f"Epoch [{epoch+1}/{epochs}] - D Loss: {d_loss.item():.4f} - G Loss: {g_loss.item():.4f}")

                avg_g_loss = np.mean(g_losses)
                avg_d_loss = np.mean(d_losses)

                # Save the best model
                if avg_g_loss < best_g_loss and avg_d_loss < best_d_loss:
                    best_g_loss = avg_g_loss
                    best_d_loss = avg_d_loss
                    best_params = {'lr': lr, 'batch_size': batch_size, 'latent_dim': latent_dim, 'epochs': epochs}
                    best_model = {'generator': gen, 'discriminator': disc}

# Save the best model
torch.save(best_model['generator'].state_dict(), os.path.join(SAVE_DIR, "best_generator.pth"))
torch.save(best_model['discriminator'].state_dict(), os.path.join(SAVE_DIR, "best_discriminator.pth"))






Using device: cpu
Running Random Search for: lr=0.0001, batch_size=8, latent_dim=64, epochs=30


Epoch 1/30: 100%|██████████| 8/8 [00:00<00:00, 22.19it/s]


Epoch [1/30] - D Loss: 0.7989 - G Loss: 0.5976


Epoch 2/30: 100%|██████████| 8/8 [00:00<00:00, 25.88it/s]


Epoch [2/30] - D Loss: 1.0318 - G Loss: 0.4442


...

Epoch [33/100] - D Loss: 0.1226 - G Loss: 2.5399


Epoch 34/100: 100%|██████████| 8/8 [00:00<00:00, 28.34it/s]


Epoch [34/100] - D Loss: 2.1857 - G Loss: 0.1040


Epoch 35/100:  38%|███▊      | 3/8 [00:00<00:00, 19.74it/s]


KeyboardInterrupt: 

In [ ]:
# Best Hyperparameters
# -----------------------
print("Best Hyperparameters:", best_params)
print("Best Generator Loss:", best_g_loss)
print("Best Discriminator Loss:", best_d_loss)

Best Hyperparameters: {'lr': 0.0001, 'batch_size': 8, 'latent_dim': 128, 'epochs': 30}
Best Generator Loss: 0.29358556131192015
Best Discriminator Loss: 2.7131193043664097
